In [1]:
from data_gen import *
from models import *
from keras import models
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM, TimeDistributed
from keras.layers import Concatenate, Flatten
from keras.layers import GRU, Conv2D, MaxPooling2D
from keras.layers import Input, Reshape
from keras.models import Model
from keras.optimizers import Adam
from keras.optimizers import RMSprop
from keras.utils.vis_utils import plot_model
import keras
import numpy as np
from autoencoder_data_gen import AEDataGen

Using TensorFlow backend.


In [2]:
da = AEDataGen()

In [16]:
gen = da.autoencoder_gen_data()
batch_size = 128
n_batches = int(len(da.gwords) * .3 / batch_size) 

In [4]:
# [x1, x2], y = next(gen)
print(len(da.wwords))

17164


In [17]:
n_batches * batch_size

5120

In [5]:
# max_word_len = 0
# for word in gofa_data_generator.words:
#     if len(word) > max_word_len:
#         max_word_len = len(word)
#         print(word)
# print(max_word_len)

In [6]:

def conv_model(n_input, n_output, n_units):
    root_word_input = Input(shape=(23, 28, 1), name="root_word_input")

    x = Conv2D(20, (5, 5), padding='same', activation='relu', name="cnn")(root_word_input)
    x = MaxPooling2D(3, 3, name="pooling")(x)

    x = Flatten(name="flatten")(x)

    state_h = Dense(n_units, activation='relu', name="cnn_encoder")(x)
    
    
    decoder_inputs = Input(shape=(None, n_output), name="target_word_input")
    decoder_gru = GRU(n_units, return_sequences=True, return_state=True, name="decoder_gru")
    decoder_outputs, _= decoder_gru(decoder_inputs, initial_state=state_h)
    
    decoder_dense = Dense(n_output, activation='softmax', name="train_output")
    decoder_outputs = decoder_dense(decoder_outputs)
    
    model = Model([root_word_input, decoder_inputs], decoder_outputs)
    encoder_model = Model(root_word_input, state_h)
    
    decoder_state_input_h = Input(shape=(n_units,))
    decoder_outputs, state_h= decoder_gru(decoder_inputs, initial_state=decoder_state_input_h)

    decoder_outputs = decoder_dense(decoder_outputs)
    decoder_model = Model([decoder_inputs, decoder_state_input_h], [decoder_outputs, state_h])

    return model, encoder_model, decoder_model

def predict(infenc, infdec, source, cardinality):
    # encode
    state = infenc.predict([source])
    # start of sequence input
    start = [0.0 for _ in range(cardinality)]
#     start[0] = 1
    target_seq = np.array(start).reshape(1, 1, cardinality)
    # collect predictions
    output = list()
    for t in range(27):
        # predict next char
        yhat, h= infdec.predict([target_seq, state])
        # store prediction
        output.append(yhat[0,0,:])
        # update state
        state = h
        # update target sequence
        target_seq = yhat
    return np.array(output)


In [7]:
n_input_length = len(da.char2int)
model, encoder_model, decoder_model = conv_model(n_input_length, n_input_length, 256)
model.compile(optimizer='adam', loss='categorical_crossentropy')

In [8]:
# model.summary()

In [9]:
history = model.fit_generator(gen, steps_per_epoch=n_batches, epochs = 20)

Epoch 1/20
93/93 [==============================] - 10s 112ms/step - loss: 1.4874
Epoch 2/20
93/93 [==============================] - 6s 68ms/step - loss: 0.9446
Epoch 3/20
93/93 [==============================] - 6s 67ms/step - loss: 0.7189
Epoch 4/20
93/93 [==============================] - 6s 67ms/step - loss: 0.5173
Epoch 5/20
93/93 [==============================] - 6s 67ms/step - loss: 0.4058
Epoch 6/20
93/93 [==============================] - 6s 68ms/step - loss: 0.3096
Epoch 7/20
93/93 [==============================] - 6s 67ms/step - loss: 0.2304
Epoch 8/20
93/93 [==============================] - 6s 67ms/step - loss: 0.1846
Epoch 9/20
93/93 [==============================] - 6s 67ms/step - loss: 0.1305
Epoch 10/20
93/93 [==============================] - 6s 68ms/step - loss: 0.0982
Epoch 11/20
93/93 [==============================] - 6s 68ms/step - loss: 0.0759
Epoch 12/20
93/93 [==============================] - 6s 66ms/step - loss: 0.0601
Epoch 13/20
93/93 [================

In [10]:

g_test_batches = int(len(da.wwords) * .3 / batch_size) 
gen3 = da.autoencoder_gen_data(batch_size=batch_size, n_batches=g_test_batches, trainset=False)
print("Train size: ", (g_test_batches * batch_size))

Train size:  5120


In [11]:
total, correct = 0, 0
in_word = 0
sims = []
for b in range(3):
    # get data from test data generator
    [X1, X2], y = next(gen3)
    for j in range(batch_size):
        root_word_matrix = X1[j].reshape((1, X1.shape[1], X1.shape[2], 1))

        # predicts the target word given root word and features
        target = predict(encoder_model, decoder_model, root_word_matrix, n_input_length)
        root = ''.join(da.one_hot_decode(X1[j]))#.replace('&', ' ')
        word = ''.join(da.one_hot_decode(y[j]))#.replace('&', ' ')
        targetS = ''.join(da.one_hot_decode(target))#.replace('&', ' ')
#         sims.append(dg.word_sim(word, targetS))
        
        # checks if the predicted and the real words are equal'
#         print(len(da.one_hot_decode(y[j])), len(da.one_hot_decode(target)))
#         print(len(dg.one_hot_decode(target)[:27]))
#         print(len(y[j]))
        if da.one_hot_decode(y[j]) == da.one_hot_decode(target)[:23]:
            correct += 1
        else:
            print(root, word.split('&')[0], '\t\t', targetS.split('&')[0])
        if root.strip() in targetS.strip():
            in_word += 1
#     print(b, root, word, targetS)
    total += batch_size
    

print('Exact Accuracy: %.2f%%' % (float(correct)/float(total)*100.0))

walalloos               walalloos 		 walaaloos
burccetoos              burccetoos 		 burcuetoos
handdaraashadinaa       handdaraashidanaa 		 handdarashatanaa
metootawsu              metootawsu 		 meeootawsu
shatimmikke             shatimmikke 		 shaametikke
zarddaoos               zarddaoos 		 zardadoos
modhdhas                mol''aas 		 modhdhaas
harshshuushas           harshshuushaas 		 harshshushays
ekkays                  ekkays 		 kkkays
inddetaasuoos           inddetaasuoos 		 inddettayooss
shuuyadinaa             shuuyidanaa 		 shuusidanaa
bundduruuqas            bundduruuqaas 		 bundduruqays
gahettoos               gayttoos 		 gahettoos
sulumuumadinaa          sulumuumidanaa 		 sulumummidanaa
kiyadinaa               kiyidanaa 		 kiyyidanaa
abaraadays              abaraadays 		 abaraabays
priixas                 priixaas 		 piixxaas
sheedhdhees             sheedhdhees 		 sheeddhdees
hiraqoos                hiraqoos 		 hiraqqoos
wodalladinaa            wodallidanaa 		 wodaalidan

In [12]:
[90.36, 88.28, 92.71, 91.15, 87.5, 92.45]

[90.36, 88.28, 92.71, 91.15, 87.5, 92.45]

In [13]:
[85.42, 71.35, 81.77, 85.42, 89.32, 87.76]

[85.42, 71.35, 81.77, 85.42, 89.32, 87.76]

In [14]:
for i in [str(x) for x in np.round(history.history['loss'], 3)]:
    print(i + ',', end=' ')

1.487, 0.945, 0.719, 0.517, 0.406, 0.31, 0.23, 0.185, 0.13, 0.098, 0.076, 0.06, 0.048, 0.039, 0.033, 0.027, 0.021, 0.021, 0.016, 0.012, 